# JUPYTER CON CODICE PER IL ROUND (1v1)

### CORE CON AGGIUNTE

In [12]:
import numpy as np
from numpy import random as npr
import matplotlib.pyplot as plt






# INITIAL DATA:_____________________________________________
cooperate = np.array( [ 1, 0 ] )
defect = np.array( [ 0, 1 ] )
# default Payoff matrix
default_R = 2
default_S = 0
default_T = 3
default_P = 1
Payoff = np.array( [ [ default_R , default_S ], [ default_T, default_P ] ] )   # matrice delle ricmopense







# STRATEGIES:________________________________________________
    # template function:
    # parameter_list = [ k, ]
    # def strategy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    #     # move either cooperate or defect
    #     move = cooperate/defect
    #     return move 

# nice guy function: always cooperate
def nice_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :      # 'None' al posto di [[]] ?
    move = cooperate
    return move 

# bad guy function: always defect 
def bad_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :        # 'None' al posto di [[]] ?
    move = defect
    return move 

# mainly nice guy: randomly defect k% of the times and cooperate 100-k %, k<50
def mainly_nice( parameter_list = [ 0.4 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    k = parameter_list[0]
    if k > 0.5 : 
        print( "Wrong function called: this is mainly_nice" )
        return - 1                                                                             # eccezione da gestire meglio?
    u = npr.random()                                                                           # corretto in random() invece di random(1)
    if ( u < k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# mainly bad guy: randomly defect k% of the times and cooperate 100-k %, k>50
def mainly_bad( parameter_list = [ 0.6 ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    k = parameter_list[ 0 ]
    if k < 0.5 : 
        print( "Wrong function called: this is mainly_bad" )
        return - 1                                                                              # eccezione da gestire meglio?
    u = npr.random()                                                                            # corretto in random() invece di random(1)
    if ( u < k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# tit_for_tat function: start by cooperating, then repeat what the opponent has done in the previous move
def tit_for_tat( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : 
        move = cooperate
    else : 
        move = match_history[ round_number - 1 ][ 1 - player_index ]
    return move

# resentful guy: cooperate until the opponent defects once, then always defect
def resentful_guy( parameter_list = [ ], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    for res in match_history :
        if np.all( res[ 1 - player_index  ] == defect ) :
            move = defect
    return move

# trusting guy (increasing odds of cooperating according to match history):
# initially is like random guy (coop/defect at k=50%), then updates k
def trusting_guy( parameter_list = [ 0.5 ], round_number = 0, match_history = [[]], player_index = 0 ) :
    # starting trust
    k = parameter_list[0]
    # environmental trust
    if ( round_number > 0 ) :
        coop = 0.         # numero di cooperazioni avversarie finora
        # 'res[ 1 - player_index  ][ 1 ]' prende il primo elemento della mossa avversaria (1 se ha cooperato, 0 se ha defectato)
        for res in match_history : coop += res[ 1 - player_index  ][ 0 ]
        k = coop / len( match_history )
    u = npr.random( 1 )
    if ( u > k ) : 
        move = defect
    else : 
        move = cooperate
    return move

# balancing guy (cooperates only for an even cooperation history, otherwise defect)
def balancing_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :
    move = defect
    if ( round_number > 0 ) :
        # total cooperation seen
        coop = 0
        for res in match_history : coop += ( res[ 0 ][ 0 ] + res[ 1 ][ 0 ])
        if ( ( coop % 2 ) == 0 ) : 
            move = cooperate
    return move

# Mid resentful: cooperates on the first move, and defects if the opponent has defected on any of the previous 3 moves, else cooperates
def mid_resentful( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0):
    move = cooperate
    # checko gli ultimi 3 turni (o meno se sono all'inizio):
    start = max(0, round_number - 3)
    for round_i in range(start, round_number):
        # controllo se la mossa avversaria corrisponde ad un defect=[0,,1]
        if np.array_equal(match_history[round_i][1 - player_index], defect):   
            move = defect
            break  # non serve continuare il controllo, esco dal for
    return move

# dictionary of all strategies
strategies = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'MainlyBad' : mainly_bad,
    'TitForTat' : tit_for_tat,
    'ResentfulGuy' : resentful_guy,
    'TrustingGuy' : trusting_guy,
    'Thanos' : balancing_guy,
    'MidResentful': mid_resentful,
}








# GAMES FUNCTIONS:

# function to play one round 1 vs 1 for two different* strategies players
def round( key_1, key_2, parameter_list_1, parameter_list_2, round_number, match_history, M = Payoff ) :
    # getting strategies from dictionary
    S_1 = strategies[ key_1 ]
    S_2 = strategies[ key_2 ]
    # computing next move for each player
    u_1 = S_1( parameter_list_1, round_number, match_history, 0 )
    u_2 = S_2( parameter_list_2, round_number, match_history, 1 )
    # computing rewards for each player
    r_1 = np.dot( u_1, Payoff.dot(u_2))
    r_2 = np.dot( u_2, Payoff.dot(u_1))
    # updating match history
    match_history.append( [ u_1, u_2 ] )
    # returning results
    return r_1, r_2



# function to play a match of N rounds 1 vs 1 for two different* strategies player
def match( key_1, key_2, parameter_list_1, parameter_list_2, N_rounds = 10, M = Payoff ) : 
    # match history strarting empty
    history = [ ]                                                                           
    # starting total rewards
    R_1 = 0
    R_2 = 0
    # list of partial sums
    Rewards = []
    for round_i in range( 0, N_rounds ) :
        r_1, r_2 = round( key_1, key_2, parameter_list_1, parameter_list_2, round_i, history, M )
        R_1 += r_1
        R_2 += r_2
        Rewards.append( [ R_1, R_2 ] )
    return Rewards              # it returns a list wiith the partial sums of the rewards

Other strategy ideas:
* detective
* random guy
* Forgiving Tit-For-Tat: starts by cooperating, then repeats what the opponent has done in the previous move, but defects only if the opponent defects for two consecutive rounds

(numero di match in una partita di n giocatori: n(n-1)/2 )

In [20]:
print( round('NiceGuy', 'MidResentful', [],[], 0, [[]]) )
print( match('MainlyNice', 'MidResentful', [0.4],[], 10) )

(2, 2)
[[2, 2], [4, 4], [6, 6], [9, 6], [9, 9], [9, 12], [9, 15], [12, 15], [12, 18], [13, 19]]
